# Start Pooling

In [53]:
#import general packages
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

#import climada stuff
from climada.entity.impact_funcs import trop_cyclone, impact_func_set
from climada.util.api_client import Client
from climada.engine import ImpactCalc

%load_ext autoreload
%autoreload 2
import exposures as ex
import grider as grd
import impact as cimp
import bound_prot_dam as bpd
import prem_ibrd as prib
import haz_int_grd as hig
import set_nominal as snom
import calc_premium as cp
import simulate_bond as sb
import pay_opt as po
import simulate_multi_cty_bond as smcb
client = Client()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define Countries and Variables for Cat Bond Development

In [17]:
#choose storm basin to load
#The STORM basins are: 'EP','NA','NI','SI','SP','WP'
#basins = ['NA','SI']

#choose country
country_1 = 174
country_2 = 662

#minimum damage to be protected per grid cell calculated using return period
lower_rp = 100

#number of simulated years
r = 10000

#set risk free rate, either single value or array
rf_rates = 0.05 

#set range of premiums 
premiums = np.arange(0.00, 0.21, 0.01) 
#inititate IBRD premium calculation function
params_ibrd = prib.init_prem_ibrd(want_plot=False)
a, k, b = params_ibrd
#set risk muliplier reported by artems
artemis_multiplier = 4.11

#set sharpe ratio to beat
requ_sharpe_ratio = 1.0

#set the maximum amount to be protected
prot_share = 0.05

## Define Cat Bond function

In [54]:
def init_sng_cty_bond(country):    
    #load tc_tracks, create hazard class and calculate exposure
    exp, applicable_basins, storm_basin_sub, tc_storms = ex.init_TC_exp(country=country, load_fls=True)
    #set up grid for cat bond
    rst_grid = grd.init_grid(exp)
    #calculate impact and aggregate impact per grid
    imp, imp_per_exp, agg_exp, imp_grid_evt = cimp.init_imp(exp, tc_storms, rst_grid) 
    #set up hazard intensity matrix per grid and event
    ws_grid = hig.init_ws_grid(tc_storms, agg_exp, 'max')
    #set principal
    nominal, tot_exp, nom_rel_exp = snom.init_nominal(impact=imp, exposure=exp, prot_share=prot_share)
    #calculate damage return period and set small damages, not intended to protect to zero
    rp_dam_grid, dam_rp_per_grid, imp_grid_evt_flt = bpd.init_dam_ret_per_grid(imp_grid_evt, lower_rp, adj_max=False, plt_dam_rp_grid=None)
    #optimize minimum and maximum triggering wind speed per grid cell
    result, optimized_xs, optimized_ys = po.init_optimization(imp_grid_evt_flt, ws_grid, rp_dam_grid, nominal)
    #create data frame containing payment vs damage per event
    pay_dam_df = po.pay_vs_damage(imp_grid_evt_flt, optimized_xs, optimized_ys, ws_grid, rp_dam_grid, nominal, include_plot=True)
    #calculate yearly event probability
    event_probability = len(pay_dam_df) / r
    #calculate expected loss and attachment probability
    exp_loss_ann, att_prob = sb.init_exp_loss_att_prob_simulation(pay_dam_df, nominal, event_probability)
    #calculate premiums using different approaches
    premium_ibrd = prib.monoExp(exp_loss_ann*100, a, k, b)/100
    premium_artemis = exp_loss_ann * artemis_multiplier
    premium_regression = cp.calc_premium_regression(exp_loss_ann *100)/100

    print(f'The premium based on past IBRD bonds is {round(premium_ibrd*100, 3)}%')
    print(f'The premium based on the artemis multiplier is {round(premium_artemis*100, 3)}%')
    print(f'The premium based on the regression model from Chatoro et al. 2022 is {round(premium_regression*100, 3)}%')

    #simulate cat bond
    premium_simulation, returns = sb.init_bond_simulation(pay_dam_df, premiums, rf_rates, event_probability, nominal) #simulate cat bond using a Monte Carlo simulation
    #determine premium to match required sharp ratio
    requ_premiums = sb.init_requ_premium(requ_sharpe_ratio, premium_simulation, rf_rates) #calculate required premium to match minimum sharpe ratio
    sb.display_premiums(requ_premiums, requ_sharpe_ratio, rf_rates, premium_simulation)   #plot premium versus sharpe ratio

    return premium_simulation, exp_loss_ann, att_prob, requ_premiums, nominal, pay_dam_df, event_probability

### Design Cat Bond for Country 1

In [55]:
premium_simulation_662, exp_loss_ann_662, att_prob_662, requ_premiums_662, nominal_662, pay_dam_df_662, event_probability_662 = init_sng_cty_bond(country=country_2)

STORM basin of country:  NA
----------------------Loading Exposure----------------------
----------------------Generate Hazard----------------------
----------------------Initiating TC Tracks----------------------


c:\users\kaibe\climada\climada_python\climada\hazard\tc_tracks.py:308: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  exp_buffer = exposure.gdf.buffer(distance=buffer, resolution=0)
c:\Users\kaibe\miniforge3\envs\climada_env\lib\site-packages\shapely\constructive.py:181: RuntimeWarning: divide by zero encountered in buffer
  return lib.buffer(
c:\Users\kaibe\miniforge3\envs\climada_env\lib\site-packages\geopandas\geodataframe.py:204: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry colu

Number of tracks in NA basin: 1344


### Design Cat Bond for Country 2

In [ ]:
premium_simulation_174, exp_loss_ann_174, att_prob_174, requ_premiums_174, nominal_174, pay_dam_df_174, event_probability_174 = init_sng_cty_bond(country=country_1)

### Set Variables for Multi-Country Bond

In [ ]:
pay_dam_df_dic = {country_1: pay_dam_df_174,
                  country_2: pay_dam_df_662}

event_probabilities = [event_probability_174, event_probability_662]

nominal_pool = nominal_174 + nominal_662

### Create Multi-Country bond

In [ ]:
metrics_mlt_cty_bond, returns_mlt_cty_bond = smcb.init_bond_simulation(pay_dam_df_dic, premiums, rf_rates, event_probabilities, nominal_pool)

requ_premiums = sb.init_requ_premium(requ_sharpe_ratio, metrics_mlt_cty_bond, rf_rates) #calculate required premium to match minimum sharpe ratio
sb.display_premiums(requ_premiums, requ_sharpe_ratio, rf_rates, metrics_mlt_cty_bond)   #plot premium versus sharpe ratio